# Code for AISTATS submission (Reconstructing Test Labels from Noisy Loss Functions)
## Label inference with log-loss and softmax-loss on MNIST

### Experiment setup

In [1]:
import os
import json
import sys
import pandas as pd
import numpy as np
from collections import Counter
import math
import time
import warnings
warnings.filterwarnings('ignore')

np.random.seed(12345)

In [2]:
all_labels = dict() # a dictionary to hold all labels from real datasets
os.listdir('data')

['.DS_Store', 'iris.csv', 'mnist.csv', 'titanic.csv', 'satimage.csv']

### MNIST

In [3]:
mnist = pd.read_csv('data/mnist.csv')
mnist = mnist.sample(frac=1)
print(mnist.shape) 
print(Counter(mnist['class']))
mnist.head()

(70000, 785)
Counter({1: 7877, 7: 7293, 3: 7141, 2: 6990, 9: 6958, 0: 6903, 6: 6876, 8: 6825, 4: 6824, 5: 6313})


pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
58332       0       0       0       0       0       0       0       0       0   
53059       0       0       0       0       0       0       0       0       0   
36369       0       0       0       0       0       0       0       0       0   
9078        0       0       0       0       0       0       0       0       0   
4675        0       0       0       0       0       0       0       0       0   

       pixel10  ...  pixel776  pixel777  pixel778  pixel779  pixel780  \
58332        0  ...         0         0         0         0         0   
53059        0  ...         0         0         0         0         0   
36369        0  ...         0         0         0         0         0   
9078         0  ...         0         0         0         0         0   
4675         0  ...         0         0         0         0         0   

       pixel781  pixel782  pixel783  pixel784  class  
58332         0         0         0         0      9  
53059         0         0         0         0      9  
36369         0         0         0         0      9  
9078          0         0         0         0      3  
4675          0         0         0         0      8  

[5 rows x 785 columns]

In [4]:
mnist_label = mnist['class'].apply(lambda x: int(x)).tolist()
Counter(mnist_label)

Counter({9: 6958,
         3: 7141,
         8: 6825,
         0: 6903,
         2: 6990,
         6: 6876,
         5: 6313,
         4: 6824,
         7: 7293,
         1: 7877})

## utility functions in the experiments

In [5]:
# function to compute the k-ary-logloss score
def k_ary_logloss(labeling=[], prediction=np.array([[]])):
    """
    labeling: 1-d list with class labels represented by integer (starts from 0)
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    Returns the k-ary-logloss score
    """
    N = len(labeling)
    loss = 0
    for i in range(N):
        cur_pred = prediction[i, labeling[i]]
        loss -= math.log(cur_pred)
    return loss / N

In [6]:
# function to compute the sigmoid loss
def sigmoid_loss(labeling=[], prediction=np.array([[]])):
    """
    labeling: 1-d list with class labels represented by integer (starts from 0)
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    Returns the sigmoid loss score
    """
    N = len(labeling)
    loss = 0
    for i in range(N):
        cur_pred = prediction[i, labeling[i]]
        sig = 1 / (1 + np.exp(-cur_pred))
        loss -= math.log(sig)
    return loss / N

In [7]:
# function to compute the sigmoid loss
def softmax_loss(labeling=[], prediction=np.array([[]])):
    """
    labeling: 1-d list with class labels represented by integer (starts from 0)
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    Returns the sigmoid loss score
    """
    N = len(labeling)
    loss = 0
    for i in range(N):
        cur_pred = prediction[i, labeling[i]]
        cur_num = np.exp(cur_pred)
        cur_den = np.sum(np.exp(prediction[i, :]))
        cur_softmax = cur_num / cur_den
        loss -= math.log(cur_softmax)
    return loss / N

In [8]:
# wrapper loss function
def loss(labeling=[], prediction=np.array([[]]), tau=0, loss_type='logloss'):
    """
    labeling: 1-d list with class labels represented by integer (starts from 0)
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    tau: noise
    loss_type: logloss, sigmoidloss, or softmaxloss
    Returns the loss score
    """
    if loss_type == 'logloss':
        return k_ary_logloss(labeling, prediction) + tau * np.random.rand()
    if loss_type == 'sigmoidloss':
        return sigmoid_loss(labeling, prediction) + tau * np.random.rand()
    if loss_type == 'softmaxloss':
        return softmax_loss(labeling, prediction) + tau * np.random.rand()

In [9]:
# function to generate the predictions
def generate_prediction(N, K, tau_max):
    """
    N: number of data points
    K: number of classes
    tau_max: maximum noise
    Return a 2-d numpy array of predictions
    """
    prediction = np.zeros([N, K])
    for i in range(N):
        n = i + 1
        for j in range(K):
            k = j + 1
            prediction[i, j] = 2 ** ((n-1) * K + k) * N * tau_max
        
        diff = prediction[i, :] - max(prediction[i, :])
        prediction[i, :] = 3 ** diff
        prediction[i, :] /= sum(prediction[i, :])
        for j in range(K):
            prediction[i, j] = sys.float_info.min if prediction[i, j] < sys.float_info.min \
                else prediction[i, j]
    return prediction 

In [10]:
# generator for K-nary labelings
def gen_K_ary(nbits, K):
    """
    nbits: number of bits in the K-nary vector
    K: K in K-nary, e.g. K=2 is binary, K=3 is ternary
    Returns a generator for all K-nary labelings of length nbits
    """
    for i in range(K**nbits):
        bin_str = str(np.base_repr(i, K))
        l = len(bin_str)
        bin_arr = np.zeros(nbits)
        for j in range(l):
            bin_arr[nbits-j-1] = float(bin_str[l-j-1])
        yield [int(x) for x in bin_arr]

In [11]:
# the main label inference algorithm
def main_algorithm(noisy_loss, N, K, prediction, loss_type):
    """
    noisy_loss: observed loss score with noise
    N: number of data points
    K: number of classes
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    loss_type: logloss, sigmoidloss, or softmaxloss
    Returns the inferred labeling
    """
    label_infer = []
    diff_min = np.inf
    lab_generator = gen_K_ary(N, K)
    
    while True:
        try:
            label_cur = next(lab_generator) 
            loss_cur = loss(label_cur, prediction, 0, loss_type)
            diff_cur = np.abs(loss_cur - noisy_loss)
            # print(loss_cur, diff_cur)
            if diff_cur < diff_min:
                label_infer = label_cur.copy()
                diff_min = diff_cur
        except:
            break
    return label_infer

In [12]:
# the main function for simulation
def main_sim(true_labeling, N, K, tau_max, loss_type, nsims=1000):
    """
    true_labeling: ground truth labeling with length at least N
    N: number of data points
    K: number of classes
    tau_max: maximum noise
    loss_type: logloss, sigmoidloss, or softmaxloss
    nsims: number of simulations
    Returns the average accuracy and the average loss score
    """
    acc = 0
    mean_loss = 0
    for ns in range(nsims):
        labeling = list(np.random.choice(true_labeling, N))
        tau = 2 * np.exp(1) * tau_max / (np.exp(1) - 1)
        prediction = generate_prediction(N, K, tau)
        noisy_loss = loss(labeling, prediction, tau_max, loss_type)
        mean_loss += noisy_loss
        labeling_infer = main_algorithm(noisy_loss, N, K, prediction, loss_type)
        all_same = (sum([x == y for x, y in zip(labeling_infer, labeling)]) == N)
        acc += all_same
    return acc / nsims, mean_loss / nsims

## label inference experiments

In [13]:
# logloss experiment
loss_type = 'logloss'
nsims = 1000
result_logloss = []

K = 10

for tau_max in [1e-4, 1e-2, 1e-1, 1]:
    for N in range(1, 16):
        try:
            start = time.time()
            acc, mean_loss = main_sim(mnist_label, N, K, tau_max, loss_type, nsims)
            seconds = time.time() - start
            result_logloss.append((K, tau_max, N, acc, mean_loss, seconds))
            print(K, tau_max, N, acc, mean_loss, seconds)
            if acc < 0.5: break
        except:
            break

10 0.0001 1 1.0 2.302074960893193 4.41563606262207
10 0.0001 2 0.681 288.676225659123 4.994109869003296
10 0.0001 3 0.039 420.6399657495326 14.392202854156494
10 0.01 1 1.0 28.470794432160627 4.123140096664429
10 0.01 2 0.17 352.7199877440206 4.94179105758667
10 0.1 1 1.0 287.73237175940136 4.196919918060303
10 0.1 2 0.162 610.9065615539575 4.909435033798218
10 1 1 0.199 638.7603199634361 4.30322003364563


In [14]:
df_logloss = pd.DataFrame(result_logloss)
df_logloss.columns = ['K', 'tau_max', 'N', 'acc', 'mean_loss', 'seconds_1k']
df_logloss['ms'] = df_logloss['seconds_1k'] # since 1000 repetitions
df_logloss.to_csv('logloss_singlequery_mnist.csv', index=False)
print(df_logloss.shape)
df_logloss.head(10)

(8, 7)


K  tau_max  N    acc   mean_loss  seconds_1k         ms
0  10   0.0001  1  1.000    2.302075    4.415636   4.415636
1  10   0.0001  2  0.681  288.676226    4.994110   4.994110
2  10   0.0001  3  0.039  420.639966   14.392203  14.392203
3  10   0.0100  1  1.000   28.470794    4.123140   4.123140
4  10   0.0100  2  0.170  352.719988    4.941791   4.941791
5  10   0.1000  1  1.000  287.732372    4.196920   4.196920
6  10   0.1000  2  0.162  610.906562    4.909435   4.909435
7  10   1.0000  1  0.199  638.760320    4.303220   4.303220

In [15]:
# softmax loss experiment
loss_type = 'softmaxloss'
nsims = 1000
result_softmaxloss = []

K = 10 
for tau_max in [1e-4, 1e-2, 1e-1, 1]:
    for N in range(1, 16):
        try:
            start = time.time()
            acc, mean_loss = main_sim(mnist_label, N, K, tau_max, loss_type, nsims)
            seconds = time.time() - start
            result_softmaxloss.append((K, tau_max, N, acc, mean_loss, seconds))
            print(K, tau_max, N, acc, mean_loss, seconds)
            if acc < 0.5: break
        except:
            break

10 0.0001 1 0.873 2.302997316019809 4.571579933166504
10 0.0001 2 0.17 2.327196636991713 6.9824042320251465
10 0.01 1 0.199 2.364116544859114 4.377845048904419
10 0.1 1 0.176 2.4239657851202097 4.173757076263428
10 1 1 0.149 2.8491950349766335 4.165590047836304


In [16]:
df_softmaxloss = pd.DataFrame(result_softmaxloss)
df_softmaxloss.columns = ['K', 'tau_max', 'N', 'acc', 'mean_loss', 'seconds_1k']
df_softmaxloss['ms'] = df_softmaxloss['seconds_1k'] # since 1000 repetitions
df_softmaxloss.to_csv('softmaxloss_singlequery_mnist.csv', index=False)
print(df_softmaxloss.shape)
df_softmaxloss.head(10)

(5, 7)


K  tau_max  N    acc  mean_loss  seconds_1k        ms
0  10   0.0001  1  0.873   2.302997    4.571580  4.571580
1  10   0.0001  2  0.170   2.327197    6.982404  6.982404
2  10   0.0100  1  0.199   2.364117    4.377845  4.377845
3  10   0.1000  1  0.176   2.423966    4.173757  4.173757
4  10   1.0000  1  0.149   2.849195    4.165590  4.165590